by Yigao Li

# Exercise 1  
(a)  

In [8]:
import numpy as np
import numpy.random as rd
from sklearn.datasets import load_breast_cancer

def chain_rule(Dg, Df, var_shape):
    # Computes the Jacobian D (g o f)
    dim = len(var_shape)
    Dg_axes = list(range(Dg.ndim-dim, Dg.ndim))
    Df_axes = list(range(dim))
    return np.tensordot(Dg, Df, axes=(Dg_axes, Df_axes))

# Compute the Jacobian blocks of X @ W + b

def DX_affine(X, W, b):
    # (d_{x_{i, j}} (X @ W))_{a, b} = e_a^T e_ie_j^T W e_b, so a,i slices equal W.T
    D = np.zeros((X.shape[0], W.shape[1], X.shape[0], X.shape[1]))
    for k in range(X.shape[0]):
        D[k,:,k,:]=W.T
    return D, X.shape

def DW_affine(X, W, b):
    # (d_{w_{i, j}} (X @ W))_{a, b} = e_a^T X e_ie_j^T e_b, so b, j slices equal x
    D = np.zeros((X.shape[0], W.shape[1], W.shape[0], W.shape[1]))
    for k in range(W.shape[1]):
        D[:,k,:,k]=X
    return D, W.shape

def Db_affine(X, W, b):
    # (d_{b_i} (1 @ b))_{a, b} = e_a^T 1 e_i^T e_b, so b, i slices are all ones
    D = np.zeros((X.shape[0], W.shape[1], b.shape[1]))
    for k in range(b.shape[1]):
        D[:,k,k]=1
    return D, b.shape
    
def logit(z):
    # This is vectorized
    return 1/(1+np.exp(-z))

def Dlogit(Z):
    # The Jacobian of the matrix logit
    D = np.zeros((Z.shape[0], Z.shape[1], Z.shape[0], Z.shape[1]))
    A = logit(Z) * logit(-Z)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            D[i,j,i,j] = A[i,j]
    return D, Z.shape

def softmax(z):
    v = np.exp(z)
    return v / np.sum(v)

def matrix_softmax(Z):
    return np.apply_along_axis(softmax, 1, Z)

def Dmatrix_softmax(Z):
    D = np.zeros((Z.shape[0], Z.shape[1], Z.shape[0], Z.shape[1]))
    for k in range(Z.shape[0]):
        v = np.exp(Z[k,:])
        v = v / np.sum(v)
        D[k,:,k,:] = np.diag(v) - np.outer(v,v)
        #print(D[k,:,k,:])
    return D, Z.shape

def cross_entropy(P, Q):
    return -np.sum(P * np.log(Q))/P.shape[0]

def DQcross_entropy(P, Q):
    return - P * (1/Q)/P.shape[0], Q.shape

def nn_loss_closure(X, Y):
    # vars[0]=W_1, vars[1]=b_1, vars[2]=W_2, vars[3]=b_2
    # cross_entropy(Y, matrix_softmax(affine(logit(affine(X; W_1, b_1))); W_2, b_2))
    def f(var):
        return cross_entropy(Y, matrix_softmax(logit((logit((X @ var[0]) + var[1]) @ var[2]) + var[3]) @ var[4] + var[5]))
    return f

def nn_loss_gradient_closure(X, Y):
    def df(var):
        # Activation of first layer
        Z1 = (X @ var[0]) + var[1]
        X2 = logit(Z1)
        
        # Activation of second layer
        Z2 = (X2 @ var[2]) + var[3]
        X3 = matrix_softmax(Z2)
        
        Z3 = (X3 @ var[4]) + var[5]
        Q = matrix_softmax(Z3)

        D_Q, Qshape = DQcross_entropy(Y, Q)
        D_Z3, Z3shape = Dmatrix_softmax(Z3)
        back_prop3 = chain_rule(D_Q, D_Z3, Qshape)

        D_X3, X3shape = DX_affine(X3, var[4], var[5])
        D_W3, W3shape = DW_affine(X3, var[4], var[5])
        D_b3, b3shape = Db_affine(X3, var[4], var[5])
        
        D_Z2, Z2shape = Dlogit(Z2)
        back_prop2 = chain_rule(chain_rule(back_prop3, D_X3, X3shape), D_Z2, Z2shape)
        
        D_X2, X2shape = DX_affine(X2, var[2], var[3])
        D_W2, W2shape = DW_affine(X2, var[2], var[3])
        D_b2, b2shape = Db_affine(X2, var[2], var[3]) 
        
        D_Z1, Z1shape = Dlogit(Z1)
        back_prop1 = chain_rule(chain_rule(back_prop2, D_X2, X2shape), D_Z1, Z1shape)

        # Jacobians for phi_1
        D_W1, W1shape = DW_affine(X, var[0], var[1])
        D_b1, b1shape = Db_affine(X, var[0], var[1])
        
        # Compute all the gradients
        W1grad = chain_rule(back_prop1, D_W1, W1shape)
        b1grad = chain_rule(back_prop1, D_b1, b1shape)
        W2grad = chain_rule(back_prop2, D_W2, W2shape)
        b2grad = chain_rule(back_prop2, D_b2, b2shape)
        W3grad = chain_rule(back_prop3, D_W3, W3shape)
        b3grad = chain_rule(back_prop3, D_b3, b3shape)
        
        return [W1grad, b1grad, W2grad, b2grad, W3grad, b3grad]
    return df

def update_blocks(x,y,t):
    # An auxiliary function for backtracking with blocks of variables
    num_blocks = len(x)
    z = [None]*num_blocks
    for i in range(num_blocks):
        z[i] = x[i] + t*y[i]
    return z
                           
def block_backtracking(x0, f, dx, df0, alpha=0.1, beta=0.5, verbose=False):
    num_blocks = len(x0)
    
    delta = 0
    for i in range(num_blocks):
        delta = delta + np.sum(dx[i] * df0[i])
    delta = alpha * delta
    
    f0 = f(x0)
    
    t = 1
    x = update_blocks(x0, dx, t)
    fx = f(x)
    while (not np.isfinite(fx)) or f0+t*delta<fx:
        t = beta*t
        x = update_blocks(x0, dx, t)
        fx = f(x)
        
    if verbose:
        print((t, delta))
        l=-1e-5
        u=1e-5
        s = np.linspace(l, u, 64)
        fs = np.zeros(s.size)
        crit = f0 + s*delta
        tan = f0 + s*delta/alpha
        for i in range(s.size):
            fs[i] = f(update_blocks(x0, dx, s[i]))
        plt.plot(s, fs)
        plt.plot(s, crit, '--')
        plt.plot(s, tan, '.')
        plt.scatter([0], [f0])
        plt.show()
            
    return x, fx

def negate_blocks(x):
    # Helper function for negating the gradient of block variables
    num_blocks = len(x)
    z = [None]*num_blocks
    for i in range(num_blocks):
        z[i] = -x[i]
    return z

def block_norm(x):
    num_blocks=len(x)
    z = 0
    for i in range(num_blocks):
        z = z + np.sum(x[i]**2)
    return np.sqrt(z)

def random_matrix(shape, sigma=0.1):
    # Helper for random initialization
    return np.reshape(sigma*rd.randn(shape[0]*shape[1]), shape)

### Begin gradient descent example

### Random seed
rd.seed(1234)

data = load_breast_cancer() # Loads the Wisconsin Breast Cancer dataset (569 examples in 30 dimensions)

# Parameters for the data
dim_data = 30
num_labels = 2
num_examples = 569

# Parameters for training
num_train = 400

X = data['data'] # Data in rows
targets = data.target # 0-1 labels
labels = np.zeros((num_examples, num_labels))
for i in range(num_examples):
    labels[i,targets[i]]=1 # Conversion to one-hot representations

# Prepare hyperparameters of the network
hidden_nodes = 20

# Initialize variables
W1_init = random_matrix((dim_data, hidden_nodes))
b1_init = np.zeros((1, hidden_nodes))

W2_init = random_matrix((hidden_nodes, hidden_nodes))
b2_init = np.zeros((1, hidden_nodes))

W3_init = random_matrix((hidden_nodes, num_labels))
b3_init = np.zeros((1, num_labels))

x = [W1_init, b1_init, W2_init, b2_init, W3_init, b3_init]
f = nn_loss_closure(X[:num_train,:], labels[:num_train,:])
df = nn_loss_gradient_closure(X[:num_train,:], labels[:num_train,:])
dx = lambda v: negate_blocks(df(v))
    
for i in range(100):
    ngrad = dx(x)
    x, fval = block_backtracking(x, f, ngrad, df(x), alpha=0.1, verbose=False)
    
    train_data = matrix_softmax(logit(logit(X[:num_train,:]@x[0] + x[1]) @ x[2] + x[3]) @ x[4] + x[5])
    train_labels = np.argmax(train_data, axis=1)
    per_correct = 100*(1 - np.count_nonzero(train_labels - targets[:num_train])/num_train)

    print("Step: %d, Avg Cross Entropy: %f, Gradient Norm: %f, Training Accuracy: %.1f percent" % (i,fval,block_norm(ngrad), per_correct))
    
test_data = matrix_softmax(logit(logit(X[num_train:,:]@x[0] + x[1]) @ x[2] + x[3]) @ x[4] + x[5])
test_labels = np.argmax(test_data, axis=1)
per_correct = 100*(1 - np.count_nonzero(test_labels - targets[num_train:])/(num_examples-num_train))

print('Final test accuracy: %.1f percent' % per_correct)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in exp


Step: 0, Avg Cross Entropy: 0.687912, Gradient Norm: 0.407086, Training Accuracy: 56.8 percent
Step: 1, Avg Cross Entropy: 0.684625, Gradient Norm: 0.049936, Training Accuracy: 56.8 percent
Step: 2, Avg Cross Entropy: 0.684061, Gradient Norm: 0.034535, Training Accuracy: 56.8 percent
Step: 3, Avg Cross Entropy: 0.684007, Gradient Norm: 0.010562, Training Accuracy: 56.8 percent
Step: 4, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 5, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 6, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 7, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 8, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 9, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 10, Avg Cross Entropy: 0.684007, Gradient No

Step: 86, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 87, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 88, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 89, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 90, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 91, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 92, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 93, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 94, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 95, Avg Cross Entropy: 0.684007, Gradient Norm: 0.004906, Training Accuracy: 56.8 percent
Step: 96, Avg Cross Entropy: 0.684007, G

(b)  
1. Less steps of backtracking. From step 5, training accuracy keeps unchanged.  
2. Backtracking layer by layer.  
3. Smaller training set.

# Exercise 2
(a)

In [29]:
import tensorflow as tf

A = tf.Variable([[1,0,1],[0,1,0],[1,0,1]], name = 'A')
B = tf.Variable([[[[0,1,0],[1,0,1],[1,1,1]],[[0,1,0],[1,0,1],[0,1,0]]],
                [[[0,1,0],[1,0,1],[0,1,0]],[[0,1,0],[1,0,1],[1,1,1]]]], name = 'B')
f = tf.tensordot(A, B, [[0,1],[2,3]])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    result = f.eval()
print(result)

[[2 0]
 [0 2]]


$$
\begin{pmatrix}a_{11}&&a_{12}&&a_{13}\\a_{21}&&a_{22}&&a_{23}\\a_{31}&&a_{32}&&a_{33}\end{pmatrix}*\begin{pmatrix}1&&0\\0&&1\end{pmatrix}
$$
can be written as
$$
\text{Contract }\mathcal{A}=\begin{pmatrix}a_{11}&&a_{12}&&a_{13}\\a_{21}&&a_{22}&&a_{23}\\a_{31}&&a_{32}&&a_{33}\end{pmatrix}
$$
$$\text{ and }\mathcal{B}=\begin{pmatrix}\begin{pmatrix}\begin{pmatrix}0&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}&&\begin{pmatrix}0&&1&&0\\1&&0&&1\\0&&1&&0\end{pmatrix}\end{pmatrix}\begin{pmatrix}\begin{pmatrix}0&&1&&0\\1&&0&&1\\0&&1&&0\end{pmatrix}&&\begin{pmatrix}0&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}\end{pmatrix}\end{pmatrix}
$$
$$
\text{ tensors over indicies }\bf{i}=\{1,2\}\text{ and }\bf{j}=\{3,4\}
$$

(b)

In [13]:
import tensorflow as tf

A = tf.Variable([[[1,0,1],[0,1,0],[1,0,1]],[[1,1,1],[1,0,1],[1,1,1]]], name = 'A')
B = tf.Variable([[[[[0,1,0],[1,0,1],[1,1,1]],[[0,1,0],[1,0,1],[0,1,0]]],
                  [[[0,1,0],[1,0,1],[0,1,0]],[[0,1,0],[1,0,1],[1,1,1]]]],
                 [[[[1,1,0],[1,0,1],[1,1,1]],[[1,1,0],[1,0,1],[1,1,1]]],
                  [[[1,1,0],[1,0,1],[1,1,1]],[[1,1,0],[1,0,1],[1,1,1]]]]], name = 'B')
f = tf.tensordot(A, B, [[1,2],[3,4]])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    result = f.eval()
print(result)

[[[[2 0]
   [0 2]]

  [[3 3]
   [3 3]]]


 [[[6 4]
   [4 6]]

  [[7 7]
   [7 7]]]]


$$
\text{Contract }\mathcal{X}=\begin{pmatrix}\begin{pmatrix}a_{11}&&a_{12}&&a_{13}\\a_{21}&&a_{22}&&a_{23}\\a_{31}&&a_{32}&&a_{33}\end{pmatrix}&&\begin{pmatrix}b_{11}&&b_{12}&&b_{13}\\b_{21}&&b_{22}&&b_{23}\\b_{31}&&b_{32}&&b_{33}\end{pmatrix}\end{pmatrix}\text{ and }\mathcal{C}=
$$
$$\begin{pmatrix}\begin{pmatrix}\begin{pmatrix}\begin{pmatrix}0&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}&&\begin{pmatrix}0&&1&&0\\1&&0&&1\\0&&1&&0\end{pmatrix}\end{pmatrix}\begin{pmatrix}\begin{pmatrix}0&&1&&0\\1&&0&&1\\0&&1&&0\end{pmatrix}&&\begin{pmatrix}0&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}\end{pmatrix}\end{pmatrix}\\ \begin{pmatrix}\begin{pmatrix}\begin{pmatrix}1&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}&&\begin{pmatrix}1&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}\end{pmatrix}\begin{pmatrix}\begin{pmatrix}1&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}&&\begin{pmatrix}1&&1&&0\\1&&0&&1\\1&&1&&1\end{pmatrix}\end{pmatrix}\end{pmatrix}\end{pmatrix}
$$
$$
\text{ tensors over indicies }\bf{i}=\{1,2\}\text{ and }\bf{j}=\{3,4\}
$$

# Project Outline  
Introduction:  
1.	What is K-means++?  
2.	Problems with traditional K-means.  
3.	Comparing with other K-means methods, advantages, why chose this one?  
4.	How does it apply or fit for our data, what to expect from the results.  
  
Clean Data:  
1.	Introduce the attributes and details about data.(Describe and summary part)  
2.	Basic analysis for attributes and plot some graphs (EDA)  
3.	Describe the problems and clean the data.  
  
K-means++ Algorithms:  
1.	Randomly choose a center point from data  
2.	For each data points, compute their distance to the center, and sum up for each center point.  
3.	Randomly choose another center points, using a weighted probability distribution to calculate a final center point.  
4.	Repeat step 2 & 3 until k center points.  
5.	Continue original K-means clustering.  
  
Normal K-means:  
1.	Introduction to general K-means.  
2.	Algorithms introduction(Steps comparing with K-means++)  
3.	Show the results  
  
Comparing two methods and results:  
1.	Difference.  
2.	Conclusion about which one is better.  
  
Conclusion:  
1.	How K-means++ improves clustering results?  
2.	What can we tell about the cluster results from the data  
3.	Problems and difficulties during the project.  
4.	More possibilities?  